gmm methods:
GMM
IVGMM
IVGMM(endog, exog, instrument, k_moms=None, k_params=None, missing='none', **kwds)
NonlinearIVGMM

In [ ]:
mdl = gmm.IV2SLS(
    endog=df['y'],
    exog=df[['x0', 'x1', 'x2']],
    instrument=df[['x0', 'x1', 'z_moderate']],
).fit()
mdl.params

In [ ]:
mdl = gmm.IVGMM(
    endog=df['y'],
    exog=df[['x0', 'x1', 'x2']],
    instrument=df[['x0', 'x1', 'z_moderate']],
).fit()
pd.Series(mdl.params, index=['x0', 'x1', 'x2'])

In [ ]:
# it can be the case that matrix above is not positive semi-definite, thus
# permute covariance matrix by changing correlation among instrument variables until it is positive definite
np.random.seed(123)
ctn = True
k = 0
kMax = 100
_params_cov = np.array([list(row[:]) for row in params['cov']])
while ctn and k < kMax:
    for i in range(len(params['cov'])):
        for j in range(len(params['cov'])):
            if(np.all(np.linalg.eigvals(params['cov']) > 0)):
                ctn = False
            if(ctn and i != j and i in [2,3,4,5] and j in [2,3,4,5]):
                params['cov'][i,j] = params['cov'][j,i] = round(np.random.uniform(),2)
            if(params['cov'][i,j] != params['cov'][j,i]):
                raise ValueError("Covariance matrix is non symmetric")
    k += 1
    if(ctn and k == kMax):
        params['cov'] = np.array([list(row[:]) for row in _params_cov])
        params['cov'][2,1] = params['cov'][1,2] = params['cov_x2_z_strong'] = round(params['cov_x2_z_strong']-0.01,2)
        k = 0
        
        print("Covariance of x2 and strongest instrument z_strong had been lowered from to", params['cov_x2_z_strong'], "in order to find p.s. matrix.")
np.random.seed(None)
if(not ctn):
    print("successfully found positive definite matrix at iteration", k-1)        
else:
    print("Covariance matrix is not positive definite!")